In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Autonomous Kid Storybook Generator for Anxiety Relief

# This Python 3 environment is set up for AI-powered story generation and image creation
# It uses Google Gemini AI for generating kid-friendly stories and Stable Diffusion for illustrations
# The system is designed to help children, including those with autism, reduce anxiety 
# by providing engaging, calming, and personalized story experiences

# The notebook combines several components:
# - Retrieval-Augmented Generation (RAG) to fetch relevant story topics
# - Sentence Transformers and FAISS for semantic search
# - Stable Diffusion for creating colorful, child-friendly images
# - Gradio for an interactive UI where kids can explore scenes and provide feedback

# Input: Children can type a place or scenario they like
# Output: The system generates a story with multiple scenes and images
# Feedback: Children can indicate if they enjoyed the story; happy feedback is saved to improve future stories

# This notebook also handles API secrets securely via Kaggle Secrets
# and supports GPU acceleration for faster image generation

In [ ]:
# 1️⃣ Install required Python libraries for AI models, embeddings, and web interface
!pip install --quiet langgraph           # Graph-based workflow library
!pip install --quiet faiss-cpu          # Library for fast similarity search
!pip install --quiet sentence-transformers  # Pretrained models for sentence embeddings
!pip install --quiet diffusers transformers accelerate safetensors  # Stable Diffusion and supporting libraries
!pip install gradio --upgrade           # Gradio library for building the web interface

# 2️⃣ Import necessary libraries for generating images with Stable Diffusion
from diffusers import StableDiffusionPipeline  # Stable Diffusion model pipeline
import torch                                   # PyTorch for tensor computations and GPU acceleration

# 3️⃣ Load the Stable Diffusion model
pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",  # Specify the pre-trained Stable Diffusion model
    torch_dtype=torch.float16           # Use half-precision (FP16) to save GPU memory and improve speed
).to("cuda")                            # Move the model to GPU for faster image generation


In [ ]:
# ---------------- IMPORTS ----------------
import os, hashlib, sqlite3, json, threading
from typing import Dict, Any
from PIL import Image
import gradio as gr
import torch
from diffusers import StableDiffusionPipeline
import google.generativeai as genai
from sentence_transformers import SentenceTransformer
import faiss
from langgraph.graph import StateGraph, END

# ---------------- CONFIG ----------------
ENABLE_SD = True
SD_MODEL_ID = "runwayml/stable-diffusion-v1-5"
SD_CACHE_PATH = "/root/.cache/huggingface/hub/models--runwayml--stable-diffusion-v1-5"
DB_PATH = "/kaggle/working/kid_feedback.db"
RAG_PATH = "/kaggle/input/safety-rules1/kid_topics_enhanced1.json"

# ---------------- GEMINI AI CONFIG ----------------
from kaggle_secrets import UserSecretsClient
try:
    GEMINI_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GEMINI_API_KEY
    genai.configure(api_key=GEMINI_API_KEY)
except Exception as e:
    raise RuntimeError(f"Authentication Error: {e}")

# ---------------- SETUP STABLE DIFFUSION ----------------
sd_pipe = None
if ENABLE_SD:
    device = "cuda" if torch.cuda.is_available() else "cpu"
    dtype = torch.float16 if device=="cuda" else torch.float32
    try:
        sd_pipe = StableDiffusionPipeline.from_pretrained(
            SD_MODEL_ID, torch_dtype=dtype, safety_checker=None
        ).to(device)
    except Exception as e:
        print(f"[ERROR] SD pipeline failed: {e}")
        sd_pipe = None

# ---------------- DATABASE ----------------
conn = sqlite3.connect(DB_PATH, check_same_thread=False)
c = conn.cursor()
c.execute("""
CREATE TABLE IF NOT EXISTS feedback (
    id INTEGER PRIMARY KEY,
    place TEXT,
    scene_number INTEGER,
    story TEXT,
    image_path TEXT,
    emotion TEXT,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
)
""")
conn.commit()

# ---------------- LOAD RAG DATA ----------------
if not os.path.exists(RAG_PATH):
    raise FileNotFoundError(f"RAG file not found: {RAG_PATH}")

with open(RAG_PATH, "r", encoding="utf-8") as f:
    rag_data = json.load(f)
topics = [t.get("topic","") for t in rag_data]

embed_model = SentenceTransformer("all-MiniLM-L6-v2")
topic_embeddings = embed_model.encode(topics, convert_to_numpy=True)
d = topic_embeddings.shape[1]
faiss.normalize_L2(topic_embeddings)
index = faiss.IndexFlatIP(d)
index.add(topic_embeddings)

# ---------------- HELPER FUNCTIONS ----------------
def rag_search(place_name: str, k=1) -> Dict[str, Any]:
    if not place_name.strip(): return {"scene_prompt": "No place provided.", "tools_prompt": ""}
    q = embed_model.encode([place_name], convert_to_numpy=True)
    faiss.normalize_L2(q)
    D,I = index.search(q,k)
    if I.size==0 or I[0][0]==-1: return {"scene_prompt": "No matching topic found.", "tools_prompt": ""}
    return rag_data[int(I[0][0])]

def generate_story(scene_desc: str, state: Dict[str, Any]) -> str:
    try:
        model = genai.GenerativeModel("gemini-2.5-flash-lite")
        prompt = f"You are a friendly children's storyteller. Scene: {scene_desc}"
        resp = model.generate_content(prompt)
        return getattr(resp,"text",str(resp)).strip()
    except Exception as e:
        state['error_flag'] = True
        state['error_message'] = f"Story generation failed: {e}"
        return ""

def auto_prompt_for_sd(scene_desc: str, state: Dict[str, Any]) -> str:
    try:
        model = genai.GenerativeModel("gemini-2.5-flash-lite")
        prompt_resp = model.generate_content(f"Convert this scene into a colorful, kid-friendly illustration: {scene_desc}")
        return getattr(prompt_resp,"text",str(prompt_resp)).strip()
    except Exception as e:
        state['error_flag'] = True
        state['error_message'] = f"SD prompt generation failed: {e}"
        return scene_desc

def sd_generate_autonomous(scene_desc: str, state: Dict[str, Any]) -> str:
    if not ENABLE_SD or sd_pipe is None: return "/kaggle/working/placeholder.png"
    prompt = auto_prompt_for_sd(scene_desc,state)
    guidance = 7.5 if "calm" in prompt.lower() else 9.0
    steps = 25 if "simple" in prompt.lower() else 40
    try:
        out = sd_pipe(prompt, num_inference_steps=steps, guidance_scale=guidance)
        img = out.images[0]
        fname = f"/kaggle/working/scene_{hashlib.md5(prompt.encode()).hexdigest()[:8]}.png"
        img.save(fname)
        return fname
    except Exception as e:
        state['error_flag'] = True
        state['error_message'] = f"SD generation failed: {e}"
        return "/kaggle/working/placeholder.png"

def db_save_scene(place: str, story: str, img: str, scene_number: int, emotion="excited"):
    c.execute(
        "INSERT INTO feedback(place,scene_number,story,image_path,emotion) VALUES (?,?,?,?,?)",
        (place, scene_number, story, img, emotion)
    )
    conn.commit()

def db_get_all_scenes(place: str):
    c.execute("SELECT story, image_path FROM feedback WHERE place=? ORDER BY scene_number", (place,))
    rows = c.fetchall()
    if rows:
        return [{"story": row[0], "image": row[1], "ready": True} for row in rows]
    return None

# ---------------- GRAPH NODES ----------------
def node_check_db(state):
    found = db_get_all_scenes(state.get("place",""))
    if found:
        state['found_in_db'] = True
        state['storybook'] = found
    else:
        state['found_in_db'] = False
    return state

def node_determine_scenes(state):
    try:
        model = genai.GenerativeModel("gemini-2.5-flash-lite")
        prompt = f"Decide how many kid-friendly story scenes to generate for {state.get('place','')}. Respond with a number."
        resp = model.generate_content(prompt)
        n = int(resp.text.strip())
        state['num_scenes'] = max(1, min(n,5))
    except:
        state['num_scenes'] = 2
    return state

def node_rag(state):
    if state.get("error_flag") or state.get("found_in_db"): return state
    state['scenes'] = []
    for _ in range(state['num_scenes']):
        rag_entry = rag_search(state.get("place",""))
        scene_prompt = rag_entry.get("scene_prompt","A fun scene")
        state['scenes'].append({'scene_desc': scene_prompt})
    return state

def node_generate_story_and_image(state):
    """Generate story + images asynchronously using readiness flags."""
    if state.get("error_flag"):
        return state

    scenes = state.get('scenes', [])
    state['storybook'] = []

    if not scenes:
        return state

    # First scene synchronous
    first = scenes[0]
    story = generate_story(first['scene_desc'], state)
    img = sd_generate_autonomous(first['scene_desc'], state)
    first['story'] = story
    first['image'] = img
    first['ready'] = True
    state['storybook'].append(first)

    # Background thread for remaining
    def generate_remaining():
        for scene in scenes[1:]:
            try:
                s = generate_story(scene['scene_desc'], state)
                i = sd_generate_autonomous(scene['scene_desc'], state)
                scene['story'] = s
                scene['image'] = i
                scene['ready'] = True
                state['storybook'].append(scene)
            except:
                scene['story'] = "⚠️ Error generating scene."
                scene['image'] = ""
                scene['ready'] = True
                state['storybook'].append(scene)

    threading.Thread(target=generate_remaining, daemon=True).start()
    return state

def node_save_if_req(state):
    return state

# ---------------- GRAPH SETUP ----------------
graph = StateGraph(dict)
graph.add_node("CHECK_DB", node_check_db)
graph.add_node("DETERMINE_SCENES", node_determine_scenes)
graph.add_node("RAG", node_rag)
graph.add_node("GENERATE", node_generate_story_and_image)
graph.add_node("SAVE_IF_REQ", node_save_if_req)
graph.set_entry_point("CHECK_DB")
graph.add_conditional_edges("CHECK_DB", lambda s: "SAVE_IF_REQ" if s.get("found_in_db") else "DETERMINE_SCENES",
                            {"SAVE_IF_REQ":"SAVE_IF_REQ","DETERMINE_SCENES":"DETERMINE_SCENES"})
graph.add_edge("DETERMINE_SCENES","RAG")
graph.add_edge("RAG","GENERATE")
graph.add_edge("GENERATE","SAVE_IF_REQ")
graph.add_edge("SAVE_IF_REQ", END)
compiled = graph.compile()

def run_graph(place: str):
    initial_state = {"place": place, "error_flag": False}
    return compiled.invoke(initial_state)

# ---------------- SCENE OUTPUT ----------------
def scene_output(state, idx):
    outputs = ""
    story_images = []
    if state.get("error_flag"):
        outputs += f"⚠️ Error: {state.get('error_message','Unknown error')}\n"

    scenes = state.get("storybook", [])
    if scenes and 0 <= idx < len(scenes):
        scene = scenes[idx]
        outputs += f"--- Scene {idx+1} ---\nStory:\n{scene.get('story','')}\n"
        if scene.get("_is_end"):
            outputs += "\n--- The End ---\n"
        img_path = scene.get("image", "")
        if img_path:
            try:
                img = Image.open(img_path)
                story_images.append(img)
            except:
                pass
    else:
        outputs += "No more scenes available.\n"

    return outputs, story_images

# ---------------- GRADIO CALLBACKS ----------------
def gradio_callback(place_name, scenes_state, idx_state):
    state = run_graph(place_name)
    scenes = state.get("storybook", [])
    current_index = 0
    outputs, story_images = scene_output({"storybook": scenes}, current_index)
    return outputs, story_images, scenes, current_index, gr.update(visible=True), gr.update(visible=True), gr.update(value=place_name)

def next_scene_callback(place_name, scenes, current_index):
    scenes = list(scenes) if scenes else []

    if not scenes:
        return "No scenes available. Generate first.", [], scenes, 0

    next_index = current_index + 1

    if next_index < len(scenes):
        if not scenes[next_index].get("ready"):
            return "⏳ Next scene is still generating, please wait...", [], scenes, current_index
        current_index = next_index
    else:
        if all(scene.get("ready") for scene in scenes):
            end_scene = {
                "scene_desc": "End of Story",
                "story": "🎉 This is the end of the story. Thank you for reading!",
                "image": "",
                "_is_end": True,
                "ready": True
            }
            scenes.append(end_scene)
            current_index = len(scenes)-1
        else:
            return "⏳ Still generating remaining scenes...", [], scenes, current_index

    outputs, story_images = scene_output({"storybook": scenes}, current_index)
    return outputs, story_images, scenes, current_index

def back_scene_callback(place_name, scenes, current_index):
    scenes = list(scenes) if scenes else []
    if not scenes:
        return "No scenes available. Generate first.", [], scenes, 0

    current_index = max(current_index - 1, 0)
    outputs, story_images = scene_output({"storybook": scenes}, current_index)
    return outputs, story_images, scenes, current_index

# ---------------- FEEDBACK ----------------
def save_feedback(place_name, scenes, mood_val, original_place_state):
    scenes = list(scenes) if scenes else []
    if mood_val == "happy":
        for idx, scene in enumerate(scenes, start=1):
            db_save_scene(place_name, scene.get("story", ""), scene.get("image", ""), idx, emotion="happy")
        msg = "Saved to DB! Thank you for your happy feedback! 🎉\nDo you want to visit a new place?"
        return gr.update(value=msg), gr.update(visible=False), gr.update(visible=False), gr.update(value=""), gr.update(visible=True), gr.update(visible=True), gr.update(value=place_name)
    else:
        msg = "Thank you for your feedback! (Not saved to DB). Would you like to try another place or regenerate images?"
        return gr.update(value=msg), gr.update(visible=False), gr.update(visible=False), gr.update(value=place_name), gr.update(visible=False), gr.update(visible=False), gr.update(value=place_name)

def visit_yes_callback(original_place):
    msg = "Great! Enter a new place and press Generate."
    return gr.update(value=msg), gr.update(value=""), gr.update(visible=False), gr.update(visible=False)

def visit_no_callback(original_place):
    msg = "Okay — place restored. Press Generate to regenerate images for the same place."
    return gr.update(value=msg), gr.update(value=original_place), gr.update(visible=False), gr.update(visible=False)

# ---------------- GRADIO UI ----------------
with gr.Blocks() as demo:
    gr.Markdown("## 📖 Autonomous Kid Storybook Generator")
    with gr.Row():
        place_input = gr.Textbox(label="Enter Place Name")
        run_btn = gr.Button("Generate Storybook")
        back_btn = gr.Button("Previous Scene")
        next_btn = gr.Button("Next Scene")

    with gr.Row():
        output_text = gr.Textbox(label="Storybook Output", lines=20)
        output_images = gr.Gallery(label="Generated Images", columns=1, height="auto")

    scenes_state = gr.State([])
    idx_state = gr.State(0)
    original_place_state = gr.State("")  

    mood = gr.Radio(label="How did you feel about this story?", choices=[("😊 Happy","happy"),("😢 Sad","sad")], visible=True)
    save_btn = gr.Button("Save my feedback", visible=False)
    visit_yes_btn = gr.Button("Yes — visit a new place", visible=False)
    visit_no_btn = gr.Button("No — keep this place", visible=False)

    run_btn.click(fn=lambda place_name, *_: gradio_callback(place_name, scenes_state, idx_state),
                  inputs=[place_input, scenes_state, idx_state],
                  outputs=[output_text, output_images, scenes_state, idx_state, mood, save_btn, place_input])

    next_btn.click(fn=next_scene_callback, inputs=[place_input, scenes_state, idx_state],
                   outputs=[output_text, output_images, scenes_state, idx_state])
    back_btn.click(fn=back_scene_callback, inputs=[place_input, scenes_state, idx_state],
                   outputs=[output_text, output_images, scenes_state, idx_state])

    save_btn.click(fn=save_feedback, inputs=[place_input, scenes_state, mood, original_place_state],
                   outputs=[output_text, mood, save_btn, place_input, visit_yes_btn, visit_no_btn, original_place_state])

    mood.change(fn=lambda m, p: gr.update(visible=True) if m else gr.update(visible=False),
                inputs=[mood, place_input], outputs=[save_btn])

    visit_yes_btn.click(fn=visit_yes_callback, inputs=[original_place_state],
                        outputs=[output_text, place_input, visit_yes_btn, visit_no_btn])
    visit_no_btn.click(fn=visit_no_callback, inputs=[original_place_state],
                       outputs=[output_text, place_input, visit_yes_btn, visit_no_btn])

demo.launch(share=True)
